In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_genai import GoogleGenerativeAI
from langchain.indexes.graph import NetworkxEntityGraph
from langchain.chains import GraphQAChain
from langchain_groq import ChatGroq
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

# from unstructured.partition.auto import partition_pdf
from langchain.docstore.document import Document
from langchain.vectorstores import Neo4jVector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import uuid
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_core.prompts.prompt import PromptTemplate
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.runnables import (
    RunnablePassthrough,
)
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA

# from langchain_experimental.graph_transformers import LLMGraphTransformerpyth

GROQ_API_KEY = "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"
load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["NEO4J_URI"] = "neo4j+s://0ff86bd3.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "NZDovxs1aUCvE9sXUk9XD23richw-sXnC0xemYYYOTM"
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
url = os.getenv("NEO4J_URI")

In [4]:
import json

data = []
with open(
    "/home/jbantupalli/LD-Agent/dataset/redial/full_synthetic_conversation_cc_redial_ultrachat_1.json",
    "r",
) as file:
    data = json.load(file)

In [5]:
full_ldagent_data = []

# Iterate through each data entry
for d in data:
    conversation_list = ""

    # Define the keys to extract conversations from
    keys = [
        "syn_trait_conv",
        "syn_reasoning_conv",
        "llmredial_trait_conv",
        "ultrachat_trait_conv",
        "cc_noisy_conv",
        "noisy_conv",
    ]

    # Extract and process conversations for each key
    for key in keys:
        # Check if the key exists in the current data entry
        if key in d:
            conversations = d[key]
            # Convert each conversation to a single string and add to the list
            for conv in conversations:
                conversation_list += conv

    # Append the processed conversations to the full data list
    full_ldagent_data.append(conversation_list)
    break
print(full_ldagent_data)

['Here is the conversation:\n\nSpeaker1: Hi, I\'m looking for some advice on a gift for someone. Do you have any ideas?\nAssistant: Of course, I\'d be happy to help! Can you tell me a bit more about the person you\'re looking to get a gift for? What\'s their age, interests, etc.?\n\nSpeaker1: Well, they\'re an adult, but that\'s about all I can say for now. I want the gift to be something they\'ll really enjoy, but I\'m having trouble coming up with ideas.\nAssistant: No worries, we can brainstorm together. Are you thinking of something practical, or something more fun and entertaining?\n\nSpeaker1: Definitely something fun. I want to bring a smile to their face when they open it.\nAssistant: That\'s great! Fun gifts are always a good idea. Can you give me a hint about what they might enjoy? For example, do they like games, or going outdoors?\n\nSpeaker1: Actually, I think they might enjoy something related to sports.\nAssistant: Ah, sports! That\'s a good lead. Are they a fan of a par

In [6]:
print(full_ldagent_data[0][:10000])

Here is the conversation:

Speaker1: Hi, I'm looking for some advice on a gift for someone. Do you have any ideas?
Assistant: Of course, I'd be happy to help! Can you tell me a bit more about the person you're looking to get a gift for? What's their age, interests, etc.?

Speaker1: Well, they're an adult, but that's about all I can say for now. I want the gift to be something they'll really enjoy, but I'm having trouble coming up with ideas.
Assistant: No worries, we can brainstorm together. Are you thinking of something practical, or something more fun and entertaining?

Speaker1: Definitely something fun. I want to bring a smile to their face when they open it.
Assistant: That's great! Fun gifts are always a good idea. Can you give me a hint about what they might enjoy? For example, do they like games, or going outdoors?

Speaker1: Actually, I think they might enjoy something related to sports.
Assistant: Ah, sports! That's a good lead. Are they a fan of a particular team or sport, o

In [41]:
Text = 'Rani Lakshmibai (or Rani Lakshmi Bai) was born on 19 November 1828, (some sources say 1835) in the town of Banares (now Varanasi) into a Marathi Karhade Brahmin family.\
 She was named Manikarnika Tambe and was nicknamed Manu.[8] Her father was Moropant Tambe[9] and her mother Bhagirathi Sapre (Bhagirathi Bai).\
  Her parents came from the Tambe village of the Guhagar taluka located in the Ratnagiri district of modern-day Maharashtra.\
 Her mother died when she was five years old. Her father was a Commander during the war of Kalyanpranth. Her father worked for Peshwa Baji Rao II of Bithoor district.\
  The Peshwa called her "Chhabili", which means "beautiful " and "lively and cheerful". \
 She was educated at home and was taught to read and write, and was more independent in her childhood than others of her age; \
 her studies included shooting, horsemanship, fencing and mallakhamba with her childhood friends Nana Sahib and Tantia Tope. \
 Rani Lakshmibai contrasted many of the patriarchal cultural expectations for women in India\'s society at this time.\
And she was known for her unique perspectives and her courage to fight against social norms even in front of the whole society.\
Rani Lakshmibai was accustomed to riding on horseback accompanied by escorts between the palace and the temple, although sometimes she was carried in a palanquin.\
Her horses included Sarangi, Pavan and Baadal; according to historians, she rode Baadal when escaping from the fort in 1858. Her palace, the Rani Mahal, has now been converted into a museum.\
 It houses a collection of archaeological remains of the period between the 9th and 12th centuries AD.'
# Define the chunk size
chunk_size = 10000

# Get the text from the first entry
Text = full_ldagent_data[0]

# Split the text into chunks
documents = [
    Document(page_content=Text[i : i + chunk_size])
    for i in range(0, len(Text), chunk_size)
]
llm = ChatGroq(
    temperature=0, model_name="llama-3.1-70b-versatile", groq_api_key=GROQ_API_KEY
)
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    # allowed_nodes=["Person", "Country", "Organization"],
    # allowed_relationships=[
    #     "NATIONALITY",
    #     "Father",
    #     "Mother",
    #     "LOCATED_IN",
    #     "WORKED_AT",
    #     "SPOUSE",
    # ],
)

graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
)
graph = NetworkxEntityGraph()
# add nodes to the graph
for node in graph_documents_filtered[0].nodes:
    graph.add_node(node.id)
# add edges to the graph
for edge in graph_documents_filtered[0].relationships:
    graph._graph.add_edge(edge.source.id, edge.target.id, relation=edge.type)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

Nodes:[Node(id='Speaker1', type='Person', properties={}), Node(id='Assistant', type='Assistant', properties={}), Node(id='Nfl The Coach Knit Hat', type='Product', properties={}), Node(id='Steelers', type='Sports team', properties={}), Node(id='Buck Knives 0750 Redpoint Serrated Folding Knife With Clip', type='Product', properties={}), Node(id='Go Girl Camo Urination Device', type='Product', properties={}), Node(id='Seavenger Zephyr 3Mm Neoprene Socks For Scuba Diving, Snorkeling, Swimming, And All Water Sports', type='Product', properties={}), Node(id='Sportula Nfl Products 3-Piece Bbq Set', type='Product', properties={}), Node(id='49Ers', type='Sports team', properties={}), Node(id='Nfl Dream Lite Pillow Pet', type='Product', properties={}), Node(id='Elite Fan Shop Ncaa Hoodie Sweatshirt Team Color Arch', type='Product', properties={}), Node(id='Champion Sports Scoop Ball Set', type='Product', properties={}), Node(id='Kijaro Xxl Dual Lock Portable Camping And Sports Chair', type='Prod

In [42]:
# graph_rag = GraphQAChain.from_llm(llm, graph=graph, verbose=True)
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph

graph = Neo4jGraph(
    url="neo4j+s://0ff86bd3.databases.neo4j.io",
    username="neo4j",
    password="NZDovxs1aUCvE9sXUk9XD23richw-sXnC0xemYYYOTM",
)
graph.add_graph_documents(
    graph_documents_filtered, baseEntityLabel=True, include_source=True
)
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)
# question = """Girl"""
# graph_rag.run(question)

In [43]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"


def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri=os.environ["NEO4J_URI"],
        auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]),
    )
    session = driver.session()
    widget = GraphWidget(graph=session.run(cypher).graph())
    widget.node_label_mapping = "id"
    return widget


showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [54]:
CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to query a graph database.
...
"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
...
"""
CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

graph_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=CYPHER_QA_PROMPT,
    verbose=True,
    allow_dangerous_requests=True,
)
# graph_chain = GraphCypherQAChain.from_llm(
#     llm, graph=graph, verbose=True, allow_dangerous_requests=True
# )

In [17]:
from langchain_community.vectorstores import Neo4jVector

# from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
hf = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
# # embeddings = OpenAIEmbeddings()

In [18]:
# embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

embeddings = hf
db = Neo4jVector.from_documents(
    documents,
    embeddings,
)

In [44]:
# graph_chain = GraphCypherQAChain.from_llm(
#     llm, graph=graph, verbose=True, allow_dangerous_requests=True
# )

In [45]:
index_name = "vector"
# keyword_index_name = "keyword"

# store = Neo4jVector.from_existing_index(
#     embeddings,
#     index_name=index_name,
#     keyword_index_name=keyword_index_name,
#     search_type="hybrid",
# )
store = Neo4jVector.from_existing_index(
    embeddings,
    url=url,
    username=username,
    password=password,
    index_name=index_name,
)


graph_chain.run("most used topic in the given dataset?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document)-[:MENTIONS]->(p) 
RETURN p, COUNT(d) as count 
ORDER BY count DESC 
LIMIT 1
Full Context:
[{'p': {'id': 'Speaker1'}, 'count': 3}]

> Finished chain.


'The most used topic in the given dataset is Speaker1, which appears 3 times.'

In [55]:
def retriever(question: str):
    # print(f"Search query: {question}")
    structured_data = graph_chain.run(question)
    unstructured_data = [el.page_content for el in store.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    print(f"Final context: {final_data}")
    return final_data

In [56]:
template = """Answer the question based only on the following context:
{context}

Question: {query}
"""
suite = ChatPromptTemplate.from_template(template)

retrieval_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | suite
    | llm
    | StrOutputParser()
)

retrieval_chain.invoke("What are the top themes in this dataset?")
retrieval_chain.invoke("Most discussed clauses?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n)-[r]->(m)
RETURN n, r, m;

Full Context:
[{'n': {'id': '54184a6856e4ea0b888e40746a53f9f2', 'text': 'Here is the conversation:\n\nSpeaker1: Hi, I\'m looking for some advice on a gift for someone. Do you have any ideas?\nAssistant: Of course, I\'d be happy to help! Can you tell me a bit more about the person you\'re looking to get a gift for? What\'s their age, interests, etc.?\n\nSpeaker1: Well, they\'re an adult, but that\'s about all I can say for now. I want the gift to be something they\'ll really enjoy, but I\'m having trouble coming up with ideas.\nAssistant: No worries, we can brainstorm together. Are you thinking of something practical, or something more fun and entertaining?\n\nSpeaker1: Definitely something fun. I want to bring a smile to their face when they open it.\nAssistant: That\'s great! Fun gifts are always a good idea. Can you give me a hint about what they might enjoy? For example, do they

'Based on the given conversation, the most discussed clauses are related to:\n\n1. Sports: Many conversations revolve around sports, including gifts for sports enthusiasts, sports-themed products, and sports-related activities.\n2. Gift-giving: Several conversations involve finding the perfect gift for someone, often with a sports theme.\n3. Product recommendations: The assistant provides recommendations for various products, including sports-themed items, and discusses their features and reviews.\n4. Customer reviews: Many conversations include customer reviews and testimonials about specific products, which helps to build trust and credibility.\n5. Health and wellness: One conversation touches on health and wellness, specifically back pain and the possibility of surgery, but this is not a dominant theme throughout the conversations.\n\nOverall, the most discussed clauses are related to sports and gift-giving, with a focus on finding the perfect product for someone who enjoys sports.'

In [57]:
retrieval_chain.invoke("What sports can I play to improve my overall fitness")
# retrieval_chain.invoke("Most discussed clauses?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n)-[r]->(m)
RETURN n, r, m;

Full Context:
[{'n': {'id': '54184a6856e4ea0b888e40746a53f9f2', 'text': 'Here is the conversation:\n\nSpeaker1: Hi, I\'m looking for some advice on a gift for someone. Do you have any ideas?\nAssistant: Of course, I\'d be happy to help! Can you tell me a bit more about the person you\'re looking to get a gift for? What\'s their age, interests, etc.?\n\nSpeaker1: Well, they\'re an adult, but that\'s about all I can say for now. I want the gift to be something they\'ll really enjoy, but I\'m having trouble coming up with ideas.\nAssistant: No worries, we can brainstorm together. Are you thinking of something practical, or something more fun and entertaining?\n\nSpeaker1: Definitely something fun. I want to bring a smile to their face when they open it.\nAssistant: That\'s great! Fun gifts are always a good idea. Can you give me a hint about what they might enjoy? For example, do they

'Based on the conversation provided, the following sports-related activities were mentioned that can be played to improve overall fitness:\n\n1. Kiteboarding (mentioned in the context of the Seavenger Zephyr 3mm Neoprene Socks)\n2. Scuba diving (mentioned in the context of the Seavenger Zephyr 3mm Neoprene Socks)\n3. Snorkeling (mentioned in the context of the Seavenger Zephyr 3mm Neoprene Socks)\n4. Swimming (mentioned in the context of the Seavenger Zephyr 3mm Neoprene Socks)\n5. Golf (mentioned in the context of an experience-based gift)\n\nAdditionally, the conversation mentions a general interest in sports, but does not provide specific recommendations for sports to improve overall fitness. However, it can be inferred that any sport or physical activity can contribute to improving overall fitness.'